In [1]:
import email.policy
from email.parser import BytesParser
import os
from concurrent.futures import ThreadPoolExecutor

In [2]:
paths = [
    "./easy_ham/easy_ham",
    "./hard_ham/hard_ham",
    "./spam_2/spam_2"
]

with open("./spam_2/spam_2/00004.bdcc075fa4beb5157b5dd6cd41d8887b", "rb") as file:
    msg = BytesParser(policy=email.policy.default).parse(file)

print("Receive")
print(msg.get_all("Received"))
print("From")
print(msg.get_all("From"))

Receive
['from mandark.labs.netnoteinc.com ([213.105.180.140]) by    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4GA3qe29480 for    <jm@jmason.org>; Thu, 16 May 2002 11:03:52 +0100', 'from webnote.net (mail.webnote.net [193.120.211.219]) by    mandark.labs.netnoteinc.com (8.11.2/8.11.2) with ESMTP id g4GA3oD28650 for    <jm@netnoteinc.com>; Thu, 16 May 2002 11:03:51 +0100', 'from webcust2.hightowertech.com (webcust2.hightowertech.com    [216.41.166.100]) by webnote.net (8.9.3/8.9.3) with ESMTP id BAA11067 for    <jm@netnoteinc.com>; Thu, 16 May 2002 01:58:00 +0100', 'from html ([199.35.236.73]) by webcust2.hightowertech.com  with    Microsoft SMTPSVC(5.5.1877.197.19); Wed, 15 May 2002 13:50:57 -0700']
From
['jordan23@mailexcite.com']


In [3]:
body = msg.get_body(preferencelist=("html"))

if body:
    payload = body.get_payload(decode=True)
    charset = body.get_content_charset() or "utf-8"
    try:
        extracted_body = payload.decode(charset, errors="replace")
    except:
        extracted_body = payload.decode("utf-8", errors="replace")

from bs4 import BeautifulSoup

clean_text = BeautifulSoup(extracted_body, "html.parser").get_text()
clean_text

'\n\n\n\n\n*****Bonus Fat Absorbers As Seen On TV, Included Free With Purchase Of 2 Or More Bottle, $24.95 Value*****\n\n\n\n***TAKE $10.00 OFF 2 & 3 MONTH SUPPLY ORDERS, $5.00 OFF 1 MONTH SUPPLY!\n***AND STILL GET YOUR BONUS!  PRICE WILL BE DEDUCTED DURING PROCESSING.\n\n\n***FAT ABSORBERS ARE GREAT FOR THOSE WHO WANT TO LOSE WEIGHT,  BUT CAN\'T STAY ON A DIET***\n\n\n***OFFER GOOD UNTIL MAY 27, 2002!  FOREIGN ORDERS INCLUDED!\n\n\n\n\nLOSE 30 POUNDS  IN 30 DAYS... GUARANTEED!!!\n\n\n\nAll Natural Weight-Loss Program, Speeds Up The Metabolism Safely\nRated #1 In Both Categories of SAFETY & EFFECTIVENESS In\n(THE United States Today)\n\nWE\'LL HELP YOU GET THINNER!\nWE\'RE GOING TO HELP YOU LOOK GOOD, FEEL GOOD AND TAKE CONTROL IN\n2002\n\n\n\n\n\n\nWhy Use Our Amazing Weight Loss Capsules?\n\n*  They act like a natural magnet to attract fat.\n*  Stimulates the body\'s natural metabolism. \n*  Controls appetite naturally and makes it easier to\n   eat the right foods consistently.\n*  

In [4]:
def extract_spam_assassin_features(file):
    with open(file, "rb") as file:
        e = BytesParser(policy=email.policy.default).parse(file)

    part = e.get_body(preferencelist=("plain", "html"))

    if part:
        payload = part.get_payload(decode=True)
        charset = part.get_content_charset()
        try:
            body = payload.decode(charset, errors="replace")
        except:
            body = payload.decode("utf-8", errors="replace")

    else:
        payload = e.get_payload(decode=True)
        body = payload.decode("utf-8", errors="replace") if payload else ""

    body = BeautifulSoup(body, "html.parser").get_text()

    return {
        "Subject": e.get("Subject"),
        "Date": e.get("Date"),
        "From": e.get_all("From"),
        "To": e.get_all("To"),
        "Received": e.get_all("Received"),
        "Content-Type": e.get_content_type(),
        "Body": body
    }    

In [5]:
overall_data = []

for p in paths:
    with ThreadPoolExecutor(max_workers=5) as executor:
        data = list(executor.map(extract_spam_assassin_features, [os.path.join(p, path) for path in os.listdir(p)]))

    overall_data.extend(data)

In [7]:
overall_data[0]

{'Subject': 'Re: New Sequences Window',
 'Date': 'Thu, 22 Aug 2002 18:26:25 +0700',
 'From': ['Robert Elz <kre@munnari.OZ.AU>'],
 'To': ['Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>'],
 'Received': ['from localhost (localhost [127.0.0.1])\tby phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36\tfor <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)',
  'from phobos [127.0.0.1]\tby localhost with IMAP (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)',
  'from listman.example.com (listman.example.com [66.187.233.211]) by    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for    <zzzz-exmh@example.com>; Thu, 22 Aug 2002 12:34:53 +0100',
  'from listman.example.com (localhost.localdomain [127.0.0.1]) by    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002    07:35:02 -0400 (EDT)',
  'from int-mx1.corp.example.com (int-mx1.corp.example.com    [172.16.52.254]) by listman.redhat.com (P

In [14]:
import pandas as pd

df = pd.DataFrame(overall_data)

df.to_csv("SpamAssassin.csv", index=False)

ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.